# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846881870075                   -0.70    4.8         
  2   -7.852321725331       -2.26       -1.53    1.0   18.0ms
  3   -7.852615414331       -3.53       -2.56    1.5   19.7ms
  4   -7.852646011307       -4.51       -2.91    2.8   93.4ms
  5   -7.852646530304       -6.28       -3.24    1.0   17.7ms
  6   -7.852646678016       -6.83       -4.00    1.0   17.6ms
  7   -7.852646686537       -8.07       -5.14    1.8   21.3ms
  8   -7.852646686727       -9.72       -5.57    2.2   23.1ms
  9   -7.852646686728      -11.81       -5.67    1.2   18.8ms
 10   -7.852646686730      -11.81       -6.75    1.0   18.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846862314257                   -0.70           4.5         
  2   -7.852553072749       -2.24       -1.64   0.80    2.2    235ms
  3   -7.852638496316       -4.07       -2.74   0.80    1.0   17.0ms
  4   -7.852646434510       -5.10       -3.33   0.80    1.8   19.8ms
  5   -7.852646679106       -6.61       -4.14   0.80    2.2   21.3ms
  6   -7.852646686379       -8.14       -4.73   0.80    1.5   19.5ms
  7   -7.852646686720       -9.47       -5.67   0.80    1.5   19.5ms
  8   -7.852646686730      -11.03       -6.59   0.80    2.0   20.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.353870e+01     3.734368e+00
 * time: 0.0506281852722168
     1     8.570129e-01     1.843102e+00
 * time: 0.24685215950012207
     2    -2.011736e+00     1.881681e+00
 * time: 0.26590919494628906
     3    -4.082721e+00     1.687418e+00
 * time: 0.29346513748168945
     4    -5.480665e+00     1.444418e+00
 * time: 0.32109999656677246
     5    -6.994651e+00     7.169183e-01
 * time: 0.3479950428009033
     6    -7.058190e+00     1.185415e+00
 * time: 0.36683011054992676
     7    -7.574520e+00     7.797221e-01
 * time: 0.38576698303222656
     8    -7.637218e+00     1.388756e+00
 * time: 0.40471315383911133
     9    -7.699479e+00     1.098589e+00
 * time: 0.4241042137145996
    10    -7.702169e+00     2.079646e+00
 * time: 0.44298410415649414
    11    -7.704987e+00     1.936835e+00
 * time: 0.46184301376342773
    12    -7.764918e+00     2.654774e-01
 * time: 0.4973900318145752
    13    -7.814602e+00     4.421393e-01
 * time: 0.5

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846850387744                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645901473                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.69s
  3   -7.852646686730      -13.27       -7.24    124ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.3619869992793995e-7
|ρ_newton - ρ_scfv| = 2.2677734682340169e-7
|ρ_newton - ρ_dm|   = 2.221724378060516e-9
